In [24]:
import os
import shutil
import pandas as pd
from rapidfuzz import process, fuzz
import unicodedata
import re

# --- CONFIGURAÇÃO ---
pasta_dfps = "/Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFP_2024/DFPs_CVM"
pasta_novo_mercado = "/Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFP_2024/Novo_Mercado"
csv_nm = "/Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/IA/DFP/empresas_novo_mercado.csv"
limiar = 70  # pontuação mínima para considerar correspondência

os.makedirs(pasta_novo_mercado, exist_ok=True)

# --- FUNÇÃO DE NORMALIZAÇÃO ---
def normalize(text):
    text = str(text).lower()
    text = re.sub(r'_versao_\d+', '', text)
    text = text.replace("_s_a", "").replace("_n_v", "")
    text = ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')
    text = re.sub(r'[^a-z0-9 ]', ' ', text)
    text = ' '.join(text.split())
    return text

# --- LEITURA CSV ---
df_nm = pd.read_csv(csv_nm)
lista_nm = df_nm["Razao_Social"].tolist()
lista_nm_norm = [normalize(n) for n in lista_nm]

# --- LISTA DE PDFS ---
pdfs = [f for f in os.listdir(pasta_dfps) if f.lower().endswith(".pdf")]

# --- LOOP DE CORRESPONDÊNCIA ---
match_count = 0
resultados = []  # lista para armazenar matches

for pdf in pdfs:
    nome_pdf_norm = normalize(pdf)
    match, score, idx = process.extractOne(nome_pdf_norm, lista_nm_norm, scorer=fuzz.token_sort_ratio)
    
    if score >= limiar:
        empresa_original = lista_nm[idx]
        src = os.path.join(pasta_dfps, pdf)
        nome_arquivo = normalize(empresa_original).replace(" ", "_") + ".pdf"
        dst = os.path.join(pasta_novo_mercado, nome_arquivo)
        shutil.copy2(src, dst)
        match_count += 1
        print(f"✅ {pdf} -> {empresa_original} ({score}%) -> {nome_arquivo}")
        
        resultados.append({
            "Arquivo_PDF": pdf,
            "Empresa_Match": empresa_original,
            "Score": score,
            "Arquivo_Destino": nome_arquivo
        })
    else:
        print(f"❌ {pdf} não corresponde a nenhuma empresa do Novo Mercado")
        resultados.append({
            "Arquivo_PDF": pdf,
            "Empresa_Match": None,
            "Score": score,
            "Arquivo_Destino": None
        })

# --- DATAFRAME FINAL ---
df_resultados = pd.DataFrame(resultados)
print("\n📌 Total de PDFs correspondentes ao Novo Mercado:", match_count)

# se quiser salvar
df_resultados.to_csv("resultados_matches.csv", index=False)

✅ minerva_s_a_versao_1.pdf -> MINERVA S.A. (72.72727272727273%) -> minerva_s_a.pdf
❌ companhia_do_metropolitano_de_sao_paulo_metro_versao_1.pdf não corresponde a nenhuma empresa do Novo Mercado
❌ cia_paulista_de_forca_e_luz_versao_1.pdf não corresponde a nenhuma empresa do Novo Mercado
❌ jbs_n_v_versao_1.pdf não corresponde a nenhuma empresa do Novo Mercado
❌ monte_rodovias_s_a_versao_1.pdf não corresponde a nenhuma empresa do Novo Mercado
❌ emae_empresa_metrop_aguas_energia_s_a_versao_3.pdf não corresponde a nenhuma empresa do Novo Mercado
❌ banrisul_armazens_gerais_sa_versao_1.pdf não corresponde a nenhuma empresa do Novo Mercado
✅ mahle_metal_leve_s_a_versao_1.pdf -> MAHLE-METAL LEVE S.A. (85.0%) -> mahle_metal_leve_s_a.pdf
❌ dtcom_direct_to_company_s_a_versao_1.pdf não corresponde a nenhuma empresa do Novo Mercado
❌ monteiro_aranha_s_a_versao_2.pdf não corresponde a nenhuma empresa do Novo Mercado
✅ simpar_s_a_versao_2.pdf -> SIMPAR S.A. (70.0%) -> simpar_s_a.pdf
❌ igua_saneamento_

In [29]:
df_nm.iloc[:60, :]

,Razao_Social,Nome_de_Pregao,Segmento,Codigo
0,AERIS IND. E COM. DE EQUIP. GERACAO DE ENERGIA...,AERIS,NM,AERI
1,AGROGALAXY PARTICIPAÇÕES S.A.,AGROGALAXY,NM,AGXY
2,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,ALLIAR,NM,AALR
3,ALLIED TECNOLOGIA S.A.,ALLIED,NM,ALLD
4,ALLOS S.A,ALLOS,NM,ALOS
5,ALLPARK EMPREENDIMENTOS PARTICIPACOES SERVICOS...,ESTAPAR,NM,ALPK
6,ALPHAVILLE S.A.,ALPHAVILLE,NM,AVLL
7,AMBIPAR PARTICIPACOES E EMPREENDIMENTOS S/A,AMBIPAR,NM,AMBP
8,AMERICANAS S.A,AMERICANAS,NM,AMER
9,ANIMA HOLDING S.A.,ANIMA,NM,ANIM


In [12]:
pdfs

['minerva_s_a_versao_1.pdf',
 'companhia_do_metropolitano_de_sao_paulo_metro_versao_1.pdf',
 'cia_paulista_de_forca_e_luz_versao_1.pdf',
 'jbs_n_v_versao_1.pdf',
 'monte_rodovias_s_a_versao_1.pdf',
 'emae_empresa_metrop_aguas_energia_s_a_versao_3.pdf',
 'banrisul_armazens_gerais_sa_versao_1.pdf',
 'mahle_metal_leve_s_a_versao_1.pdf',
 'dtcom_direct_to_company_s_a_versao_1.pdf',
 'monteiro_aranha_s_a_versao_2.pdf',
 'simpar_s_a_versao_2.pdf',
 'igua_saneamento_s_a_versao_1.pdf',
 'alpargatas_s_a_versao_3.pdf',
 'tim_s_a_versao_1.pdf',
 'gama_participacoes_s_a_versao_1.pdf',
 'dexco_s_a_versao_1.pdf',
 'quantum_participacoes_s_a_versao_1.pdf',
 'livetech_da_bahia_industria_e_comercio_s_a_versao_2.pdf',
 'wetzel_s_a_versao_1.pdf',
 'votorantim_cimentos_s_a_versao_1.pdf',
 'gsh_corp_participacoes_s_a_versao_1.pdf',
 'aeris_ind_e_com_de_equip_para_ger_de_eng_s_a_versao_3.pdf',
 'cia_catarinense_de_aguas_e_saneam_casan_versao_1.pdf',
 'general_shopping_e_outlets_do_brasil_brasil_s_a_versao_1

In [13]:
nome_pdf_norm

'alphaville pdf'